PROJECT ML - FINANCIAL PREDICITIONS

In [1]:
pip install snscrape praw beautifulsoup4 requests selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install snscrape==0.4.3.20220106

  Attempting uninstall: snscrape
    Found existing installation: snscrape 0.7.0.20230622
    Uninstalling snscrape-0.7.0.20230622:
      Successfully uninstalled snscrape-0.7.0.20230622
Note: you may need to restart the kernel to use updated packages.


scrapping donnée sentimentale sur twitter

In [7]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [13]:
import tweepy
import pandas as pd

# Remplace par tes vraies clés
bearer_token = "AAAAAAAAAAAAAAAAAAAAABt%2FyAEAAAAAkE4aqPjCgLQPMsE76AZ3FZy2hOg%3DeHnqylqh8xpeaI3v3h1UVMGMMj3HWsYnYfNw5xpDubLGUSjkZs"

client = tweepy.Client(bearer_token=bearer_token)

def get_tweets(query, max_results=100):
    tweets = client.search_recent_tweets(
        query=query,
        tweet_fields=["created_at", "public_metrics", "text", "author_id", "lang"],
        max_results=min(max_results, 100)  # limite API
    )

    tweet_data = []
    for tweet in tweets.data:
        tweet_data.append({
            'date': tweet.created_at,
            'content': tweet.text,
            'author_id': tweet.author_id,
            'like_count': tweet.public_metrics["like_count"],
            'retweet_count': tweet.public_metrics["retweet_count"]
        })

    return pd.DataFrame(tweet_data)

# Exemple : tweets récents sur $AAPL
df = get_tweets("AAPL stock", max_results=50)
print(df.head())

TooManyRequests: 429 Too Many Requests
Too Many Requests

analyse sentiments sur twitter 

In [11]:
!pip install textblob
!python -m textblob.download_corpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
[nltk_data] Downloading package brown to /Users/albane/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/albane/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /Users/albane/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/albane/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/albane/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]

In [15]:
import tweepy
import pandas as pd
from textblob import TextBlob

# --- Connexion à l'API Twitter ---
bearer_token = "AAAAAAAAAAAAAAAAAAAAABt%2FyAEAAAAAkE4aqPjCgLQPMsE76AZ3FZy2hOg%3DeHnqylqh8xpeaI3v3h1UVMGMMj3HWsYnYfNw5xpDubLGUSjkZs"
client = tweepy.Client(bearer_token=bearer_token)

# --- Fonction pour récupérer les tweets ---
def get_tweets(query, max_results=10):
    tweets = client.search_recent_tweets(
        query=query,
        tweet_fields=["created_at", "public_metrics", "text", "author_id", "lang"],
        max_results=min(max_results, 10)
    )

    tweet_data = []
    for tweet in tweets.data:
        sentiment = TextBlob(tweet.text).sentiment.polarity
        sentiment_label = "positive" if sentiment > 0 else "negative" if sentiment < 0 else "neutral"

        tweet_data.append({
            'date': tweet.created_at,
            'content': tweet.text,
            'author_id': tweet.author_id,
            'like_count': tweet.public_metrics["like_count"],
            'retweet_count': tweet.public_metrics["retweet_count"],
            'sentiment_score': sentiment,
            'sentiment': sentiment_label
        })

    return pd.DataFrame(tweet_data)

# --- Exemple d'utilisation ---
df = get_tweets("AAPL stock", max_results=50)
print(df[['content', 'sentiment_score', 'sentiment']].head())

TooManyRequests: 429 Too Many Requests
Too Many Requests

scrapping reddit

In [19]:
import praw
import pandas as pd

# --- Authentification Reddit ---
reddit = praw.Reddit(
    client_id="dJXSykDD0JZeQw4MIVaz3Q",              
    client_secret="qRBbf_fHRG4lN9wv6ZDsEYYYlrHIeg",     
    user_agent="Nearby-Comb-849"
    )         

# --- Fonction de scraping ---
def scrap_reddit(keyword, max_posts=30):
    posts = []
    for submission in reddit.subreddit("all").search(keyword, sort='new', limit=max_posts):
        posts.append({
            'title': submission.title,
            'content': submission.selftext,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'created_utc': submission.created_utc,
            'url': submission.url
        })
    return pd.DataFrame(posts)

# --- Exemple : test sur un mot-clé ---
df_reddit = scrap_reddit("AAPL stock", 30)
print(df_reddit[['title', 'score', 'num_comments']].head())

                                               title  score  num_comments
0                                  She’s a crashing!      2             0
1  $NVDA and the Magnificent Seven: Tech Rebound ...      1             0
2  Interesting Stocks Today (04/8) - Minor Market...      6             0
3  Interesting Stocks Today (04/8) - Minor Market...      4             0
4  Interesting Stocks Today (04/8) - Minor Market...      1             0


analyse sentiment 

In [20]:
!pip install textblob
!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /Users/albane/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/albane/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/albane/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/albane/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/albane/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/albane/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [21]:
import praw
import pandas as pd
from textblob import TextBlob

# --- Authentification Reddit ---
reddit = praw.Reddit(
    client_id="dJXSykbD0JZeQw4MIVaz3Q",
    client_secret="qRBbf_fHRG4IN9wv6ZDsEYYlrH1Eg",
    user_agent="sentiment_scraper_by_albane"
)

# --- Fonction avec analyse de sentiment ---
def scrap_reddit_with_sentiment(keyword, max_posts=30):
    posts = []
    for submission in reddit.subreddit("all").search(keyword, sort='new', limit=max_posts):
        text = f"{submission.title} {submission.selftext}"

        # Analyse de sentiment
        blob = TextBlob(text)
        polarity = blob.sentiment.polarity  # de -1 à +1
        sentiment = (
            "positive" if polarity > 0 else
            "negative" if polarity < 0 else
            "neutral"
        )

        posts.append({
            'title': submission.title,
            'content': submission.selftext,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'created_utc': submission.created_utc,
            'url': submission.url,
            'sentiment_score': polarity,
            'sentiment': sentiment
        })

    return pd.DataFrame(posts)

# --- Exemple ---
df_reddit = scrap_reddit_with_sentiment("AAPL stock", 30)
print(df_reddit[['title', 'sentiment_score', 'sentiment']].head())

ResponseException: received 401 HTTP response